# ※ 개인정보 보호를 위해 Output은 꼭 삭제해야합니다. ※

# ※ 개인정보 보호를 위해 Output은 꼭 삭제해야합니다. ※

# ※ 개인정보 보호를 위해 Output은 꼭 삭제해야합니다. ※

# ※ 개인정보 보호를 위해 Output은 꼭 삭제해야합니다. ※

# ※ 개인정보 보호를 위해 Output은 꼭 삭제해야합니다. ※

# ※ 개인정보 보호를 위해 Output은 꼭 삭제해야합니다. ※

# ※ 개인정보 보호를 위해 Output은 꼭 삭제해야합니다. ※

# ※ 개인정보 보호를 위해 Output은 꼭 삭제해야합니다. ※

# ※ 개인정보 보호를 위해 Output은 꼭 삭제해야합니다. ※

# ※ 개인정보 보호를 위해 Output은 꼭 삭제해야합니다. ※

In [ ]:
from pathlib import Path


CONNECTION_STRING_PATH = Path('./appsettings.Production.json')
CONNECTION_STRING_NAME = 'Database.SQL'

USERS_TABLE_NAME = 'users'
VERIFICATION_KEYS_TABLE_NAME = 'verification.keys'
EOULLIM_BOOTHS_TABLE_NAME = 'EoullimBooths'
EOULLIM_BALANCES_TABLE_NAME = 'EoullimBalances'
EOULLIM_TRANSACTIONS_TABLE_NAME = 'EoullimTransactions'
EOULLIM_PAYMENTS_TABLE_NAME = 'EoullimPayments'

In [ ]:
import pymysql, json

# ConnectionString 로드
with CONNECTION_STRING_PATH.open(encoding='utf-8') as f:
    connection_keys = {kv[0].strip().lower(): kv[1] for pair in json.load(f)['ConnectionStrings'][CONNECTION_STRING_NAME].split(';') if pair and (kv := pair.split('=')) and len(kv) == 2}
    
class Connection:
    def __enter__(self):
        self.conn = pymysql.connect(
            host=connection_keys['server'],
            port=int(connection_keys['port']),
            user=connection_keys['uid'],
            password=connection_keys['pwd'],
            db=connection_keys['database'],
            charset=connection_keys['charset'],
            cursorclass=pymysql.cursors.DictCursor
        )

        return self.conn.cursor()
    
    def __exit__(self, type, value, traceback):
        self.conn.close()

def connection():
    return Connection()

def execute(sql, params=None):
    c = connection()
    with c as cursor:
        cursor.execute(sql, params)
        c.conn.commit()

def fetch_scalar(sql, params=None):
    with connection() as cursor:
        cursor.execute(sql, params)
        item = cursor.fetchone()
        return item and item[list(item.keys())[0]]
    
def fetch_all(sql, params=None):
    with connection() as cursor:
        cursor.execute(sql, params)
        return cursor.fetchall()

In [ ]:
from typing import Any
import pandas as pd
from pandas import DataFrame, Series, ExcelWriter
from IPython.display import display

In [ ]:
users = fetch_all(f"""
    SELECT
        u.id, u.phone, vk.type, 
        vk.department, vk.grade,
        vk.classroom, vk.number,
        u.name, u.created_at
    FROM `{USERS_TABLE_NAME}` AS u
    INNER JOIN `{VERIFICATION_KEYS_TABLE_NAME}` vk ON u.id = vk.user_id
    ORDER BY u.id ASC
""")

# 미인증 사용자 집계
unverified_users = fetch_all(f"""
    SELECT
        u.id, u.phone, u.name, u.created_at
    FROM `{USERS_TABLE_NAME}` AS u
    WHERE u.id NOT IN (
        SELECT vk.user_id FROM `{VERIFICATION_KEYS_TABLE_NAME}` vk
    )
    ORDER BY u.id ASC
""")

if unverified_users:
    print('미인증 사용자가 존재합니다!!')
    print('id, phone, name, created_at')

for user in unverified_users:
    print(f'{user["id"]}, {user["phone"]}, {user["name"]}, {user["created_at"]}')

# 판다스로 변환
users = DataFrame(users)

def get_identifier(row: dict[str, Any]):
    if row['type'] == 'TEACHER':
        return '교직원'
    
    department = row['department'] or 'X'
    grade, classroom, number = map(lambda x: 0 if x is None else int(x), (row['grade'], row['classroom'], row['number']))

    return f'{department[:1]}{grade}{classroom}{number:02d}'

users['identifier'] = users.apply(get_identifier, axis=1)

# 개인정보 보호를 위해 전화번호 마스킹
users['phone'] = users['phone'].apply(lambda x: f'{x[:3]}xxxx{x[7:]}')

# 필요한 컬럼만 추출
users = users[['id', 'phone', 'identifier', 'name', 'created_at']]
users.rename(columns={
    'id': 'ID',
    'phone': '전화번호',
    'identifier': '식별번호',
    'name': '이름',
    'created_at': '가입일'
}, inplace=True)
users.set_index('ID', inplace=True)

display(users.head())

In [ ]:
booths = fetch_all(f"""
    SELECT
        id,
        class,
        name
    FROM `{EOULLIM_BOOTHS_TABLE_NAME}`
    ORDER BY id ASC
""")

booths = DataFrame(booths)

booths.rename(columns={
    "id": "ID",
    "class": "분류",
    "name": "이름"
}, inplace=True)

booths.set_index('ID', inplace=True)

display(booths.head())

In [ ]:
balances = fetch_all(f"""
    SELECT
        id,
        userId,
        boothId,
        amount,
        type
    FROM `{EOULLIM_BALANCES_TABLE_NAME}`
    ORDER BY id ASC
""")

balances = DataFrame(balances)
balances = balances.astype({
    'userId': 'Int64',
    'boothId': 'Int64',
    'amount': 'Int64',
})

personal_balances = balances[balances['type'] == 'personal']
booth_balances = balances[balances['type'] == 'booth']

# 링크되지 않은 테이블 집계
unlinked_balances = pd.concat([
    personal_balances[personal_balances['userId'].isna()],
    booth_balances[booth_balances['boothId'].isna()]
])

if unlinked_balances.size > 0:
    print('링크되지 않은 테이블이 존재합니다!!')
    print('id, amount, type')

for balance in unlinked_balances.to_dict(orient='records'):
    print(f'{balance["id"]}, {balance["amount"]}, {balance["type"]}')

# 링크되지 않은 테이블 제거
personal_balances = personal_balances[personal_balances['userId'].notna()]
booth_balances = booth_balances[booth_balances['boothId'].notna()]

personal_balances = personal_balances[['id', 'userId', 'amount']]
booth_balances = booth_balances[['id', 'boothId', 'amount']]

column_names = {
    'id': 'ID',
    'userId': '사용자ID',
    'boothId': '부스ID',
    'amount': '잔액'
}

personal_balances.rename(columns=column_names, inplace=True)
booth_balances.rename(columns=column_names, inplace=True)

personal_balances.set_index('ID', inplace=True)
booth_balances.set_index('ID', inplace=True)

display(personal_balances.head())
display(booth_balances.head())

In [ ]:
payments = fetch_all(f"""
    SELECT
        id,
        userId,
        boothId,
        paidAmount,
        refundedAmount,
        paymentTransactionId,
        refundTransactionId,
        paymentComment,
        refundComment,
        status,
        paidTime,
        refundedTime
    FROM {EOULLIM_PAYMENTS_TABLE_NAME}
    ORDER BY id ASC
""")

payments = DataFrame(payments)
payments = payments.astype({
    'userId': 'Int64',
    'boothId': 'Int64',
    'paidAmount': 'Int64',
    'refundedAmount': 'Int64',
    'paymentTransactionId': 'Int64',
    'refundTransactionId': 'Int64',
})

paid_payments = payments[[
    'id',
    'userId',
    'boothId',
    'paidAmount',
    'paymentTransactionId',
    'paymentComment',
    'paidTime'
]].copy()
refunded_payments = payments[payments['status'] == 'refunded'][[
    'id',
    'userId',
    'boothId',
    'refundedAmount',
    'refundTransactionId',
    'refundComment',
    'refundedTime'
]]

column_names = {
    'id': 'ID',
    'userId': '사용자ID',
    'boothId': '부스ID',
    'paidAmount': '결제금액',
    'refundedAmount': '환불금액',
    'paymentTransactionId': '결제트랜잭션ID',
    'refundTransactionId': '환불트랜잭션ID',
    'paymentComment': '결제사유',
    'refundComment': '환불사유',
    'paidTime': '결제시간',
    'refundedTime': '환불시간'
}

paid_payments.rename(columns=column_names, inplace=True)
refunded_payments.rename(columns=column_names, inplace=True)

paid_payments.set_index('ID', inplace=True)
refunded_payments.set_index('ID', inplace=True)

display(paid_payments.head())
display(refunded_payments.head())

In [ ]:
transactions = fetch_all(f"""
    SELECT
        id,
        senderId,
        receiverId,
        amount,
        comment,
        time
    FROM {EOULLIM_TRANSACTIONS_TABLE_NAME}
    ORDER BY id ASC
""")

transactions = DataFrame(transactions)
transactions = transactions.astype({
    'senderId': 'Int64',
    'receiverId': 'Int64',
    'amount': 'Int64',
})

default_transactions = transactions[~transactions['senderId'].isna()].copy()
exchange_transactions = transactions[transactions['senderId'].isna()].drop(columns=['senderId'])

column_names = {
    'id': 'ID',
    'senderId': '송금자잔고ID',
    'receiverId': '수신자잔고ID',
    'amount': '금액',
    'comment': '사유',
    'time': '시간'
}

default_transactions.rename(columns=column_names, inplace=True)
exchange_transactions.rename(columns=column_names, inplace=True)

default_transactions.set_index('ID', inplace=True)
exchange_transactions.set_index('ID', inplace=True)

display(default_transactions.head())
display(exchange_transactions.head())

In [ ]:
balance_refunds = paid_payments[paid_payments["부스ID"] == 1].copy()
balance_refunds_sum = balance_refunds["결제금액"].sum()
# 1000원 단위 절사
balance_refunds["결제금액"] = balance_refunds["결제금액"] // 1000 * 1000
# 10월 이후 결제금액
balance_refunds = balance_refunds[balance_refunds["결제시간"] >= "2023-10-01 00:00:00"]

balance_refunds["식별번호"] = balance_refunds["사용자ID"].apply(lambda x: users.loc[x]["식별번호"])
balance_refunds["이름"] = balance_refunds["사용자ID"].apply(lambda x: users.loc[x]["이름"])

balance_refunds = balance_refunds[['사용자ID', '식별번호', '이름', '결제금액', '결제시간', '결제트랜잭션ID']]
display(balance_refunds.head())

# 절사된 금액 합계
print(f"절사잔액 합계: {balance_refunds_sum - balance_refunds['결제금액'].sum()}원")

In [ ]:
tables = {
    '사용자': users,
    '부스': booths,
    '개인잔고': personal_balances,
    '부스잔고': booth_balances,
    '결제내역': paid_payments,
    '환불내역': refunded_payments,
    '기본거래내역': default_transactions,
    '환전거래내역': exchange_transactions,
    '잔액환불': balance_refunds
}

tables = {name: table.copy() for name, table in tables.items()}

with ExcelWriter("../한움페이통계데이터.xlsx", engine='xlsxwriter') as writer:
    # 잔액 필드 추가
    tables['사용자']['잔액'] = tables['사용자'].apply(lambda row: tables['개인잔고'][tables['개인잔고']['사용자ID'] == row.name]['잔액'].sum(), axis=1)
    tables['부스']['잔액'] = tables['부스'].apply(lambda row: tables['부스잔고'][tables['부스잔고']['부스ID'] == row.name]['잔액'].sum(), axis=1)

    for name, table in tables.items():
        table.to_excel(writer, sheet_name=name)
